In [1]:
import tarfile
import urllib.request
import tempfile
import anndata as ad
import scanpy as sc

import pandas as pd
import numpy as np
import seaborn as sb
from scipy import io
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import pickle
import copy

import rpy2.rinterface_lib.callbacks
import logging
from rpy2.robjects import pandas2ri
%load_ext rpy2.ipython

In [2]:
%%R
old_paths <- .libPaths()[-1]
new_paths <- c("~/R/nips", old_paths)
.libPaths(new_paths)

suppressMessages(library("dplyr"))
suppressMessages(library("Seurat"))
suppressMessages(library("anndata"))
suppressMessages(library("SingleCellExperiment"))
suppressMessages(library("scran"))
suppressMessages(library("Matrix"))

In [3]:
## VIASH START

par = {
    "id": "babel_GM12878",
    "input": "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE166797&format=file",
    "organism": "human",
    "output_rna": "output_rna.h5ad",
    "output_mod2": "output_mod2.h5ad",
    "rep_n": 1
}

## VIASH END

In [4]:
tempdir = '/home/alina/Desktop/neurips/data/babel_data'

print("Reading sample h5")
ad_rep1 = sc.read_10x_h5(tempdir + '/GSM5085810_GM12878_rep1_filtered_feature_bc_matrix.h5', gex_only = False)
ad_rep2 = sc.read_10x_h5(tempdir + '/GSM5085812_GM12878_rep2_filtered_feature_bc_matrix.h5', gex_only = False)

ad_rep1.var_names_make_unique()
ad_rep2.var_names_make_unique()

print("Merging into one AnnData object")
comb = ad.concat(
    {"rep1": ad_rep1, "rep2": ad_rep2},
    axis=0,
    join="outer",
    label="batch",
    fill_value=0,
    index_unique="-"
)
catvar = pd.concat([ad_rep1.var, ad_rep2.var]).drop_duplicates()
comb.var = catvar.loc[comb.var_names]

print("Splitting up into GEX and ATAC")
adata_RNA = comb[:, comb.var.feature_types == "Gene Expression"].copy()
adata_ATAC = comb[:, comb.var.feature_types == "Peaks"].copy()

adata_RNA.var.feature_types = 'GEX'
adata_ATAC.var.feature_types = 'ATAC'

uns = { "dataset_id" : par["id"], "organism" : par["organism"] }
adata_RNA.uns = uns
adata_ATAC.uns = uns

print("Writing output")
adata_RNA.write_h5ad(par['output_rna'], compression = "gzip")
adata_ATAC.write_h5ad(par['output_mod2'], compression = "gzip")

Reading sample h5


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


Merging into one AnnData object
Splitting up into GEX and ATAC


/home/alina/Downloads/ENTER/envs/nips/lib/python3.8/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'feature_types' as categorical
/home/alina/Downloads/ENTER/envs/nips/lib/python3.8/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'genome' as categorical


Writing output


/home/alina/Downloads/ENTER/envs/nips/lib/python3.8/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'feature_types' as categorical
/home/alina/Downloads/ENTER/envs/nips/lib/python3.8/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'genome' as categorical


# Quality Control metrics

In [5]:
# percent_mito
mito_genes = adata_RNA.var_names.str.startswith('MT-')
adata_RNA.obs['pct_counts_mt'] = (np.sum(adata_RNA[:, mito_genes].X, axis=1).A1 / np.sum(adata_RNA.X, axis=1).A1)*100
adata_RNA.obs['pct_counts_mt'].max()

88.26816

In [6]:
# UMI counts per cell
# rows: n of cells = 7361; columns: n of genes = 36601
# asarray convert matrix (7361, 1) to array (7361,)
adata_RNA.obs['n_counts'] = np.asarray(np.sum(adata_RNA.X, axis = 1)).reshape(-1)
adata_RNA.obs['n_counts'].min()

1.0

In [7]:
# n of genes per cell
adata_RNA.obs['n_genes'] = np.asarray((adata_RNA.X > 0).sum(axis=1)).reshape(-1)
adata_RNA.obs['n_genes'].max()

9421

In [8]:
# Filter cells according to identified QC thresholds:

print('Total number of cells: {:d}'.format(adata_RNA.n_obs))

sc.pp.filter_cells(adata_RNA, min_counts = 1500)

print('Number of cells after min count filter: {:d}'.format(adata_RNA.n_obs))

sc.pp.filter_cells(adata_RNA, max_counts = 40000)
print('Number of cells after max count filter: {:d}'.format(adata_RNA.n_obs))

adata_RNA_filtered = adata_RNA[adata_RNA.obs['pct_counts_mt'] < 20]
print('Number of cells after MT filter: {:d}'.format(adata_RNA.n_obs))

sc.pp.filter_cells(adata_RNA, min_genes = 700)
print('Number of cells after gene filter: {:d}'.format(adata_RNA.n_obs))

Total number of cells: 7361
Number of cells after min count filter: 6773
Number of cells after max count filter: 6653
Number of cells after MT filter: 6653
Number of cells after gene filter: 6613


In [9]:
# Filter genes

print('Total number of genes: {:d}'.format(adata_RNA.n_vars))
sc.pp.filter_genes(adata_RNA, min_cells=20)
print('Number of genes after cell filter: {:d}'.format(adata_RNA.n_vars))
adata_RNA.write('./adata_RNA_filtered.h5ad')

Total number of genes: 36601
Number of genes after cell filter: 15970


# Normalization with PCA (size factors)

In [10]:
adata_RNA_pca = adata_RNA.copy()

In [11]:
# Perform a clustering for scran normalization in clusters

adata_pp = adata_RNA_pca.copy()

sc.pp.normalize_per_cell(adata_pp, counts_per_cell_after=1e6)
sc.pp.log1p(adata_pp)
sc.pp.pca(adata_pp, n_comps=50)
sc.pp.neighbors(adata_pp)
sc.tl.louvain(adata_pp, key_added='groups', resolution=0.5)

In [12]:
# Preprocess variables for scran normalization

input_groups_pca = adata_pp.obs['groups']
del adata_pp
data_mat_pca = adata_RNA_pca.X.T
io.mmwrite("data_mat_pca.mtx", data_mat_pca, comment='', field=None, precision=None, symmetry=None)

In [13]:
%%R -i input_groups_pca -o size_factors_pca

data_mat_pca <- readMM("./data_mat_pca.mtx")
size_factors_pca = calculateSumFactors(data_mat_pca, clusters=input_groups_pca, min.mean=0.1)

In [14]:
# Keep the count data in a counts layer
adata_RNA_pca.layers["counts"] = adata_RNA_pca.X.copy()

In [15]:
# Normalize & Log-transform 
adata_RNA_pca.obs['size_factors'] = size_factors_pca
adata_RNA_pca.X /= adata_RNA_pca.obs['size_factors'].values[:,None]
adata_RNA_pca.layers["log_norm"] = sc.pp.log1p(adata_RNA_pca.X)

# Normalization without PCA (size factors)

In [16]:
# Perform a clustering for scran normalization in clusters

adata_pp = adata_RNA.copy()

sc.pp.normalize_per_cell(adata_pp, counts_per_cell_after=1e6)
sc.pp.log1p(adata_pp)
sc.pp.neighbors(adata_pp)
sc.tl.louvain(adata_pp, key_added='groups', resolution=0.5)

         Falling back to preprocessing with `sc.pp.pca` and default params.


In [17]:
# Preprocess variables for scran normalization

input_groups = adata_pp.obs['groups']
del adata_pp
data_mat = adata_RNA.X.T
io.mmwrite("data_mat.mtx", data_mat, comment='', field=None, precision=None, symmetry=None)

In [18]:
%%R -i input_groups -o size_factors

data_mat <- readMM("./data_mat.mtx")
size_factors = calculateSumFactors(data_mat, clusters=input_groups, min.mean=0.1)

In [19]:
# Keep the count data in a counts layer
adata_RNA.layers["counts"] = adata_RNA.X.copy()

In [20]:
# Normalize & Log-transform 
adata_RNA.obs['size_factors'] = size_factors
adata_RNA.X /= adata_RNA.obs['size_factors'].values[:,None]
adata_RNA.layers["log_norm"] = sc.pp.log1p(adata_RNA.X)

# Integration

In [32]:
# with PCA
adata_RNA_pca.obs['size_factors']

AAACAGCCAAGCGATG-1-rep1    1.246474
AAACATGCACACAATT-1-rep1    1.304942
AAACCGAAGTTAACCA-1-rep1    1.424063
AAACCGCGTCAAGTAT-1-rep1    0.638786
AAACCGGCACCATATG-1-rep1    0.111328
                             ...   
TTTGTGAAGGTCAAAG-1-rep2    2.913116
TTTGTGGCAATAATGG-1-rep2    2.834000
TTTGTGGCATTGTGCA-1-rep2    0.847679
TTTGTGTTCGAGGTGG-1-rep2    1.751037
TTTGTGTTCGCATCCT-1-rep2    1.431803
Name: size_factors, Length: 6613, dtype: float64

In [33]:
# without PCA
adata_RNA.obs['size_factors']

AAACAGCCAAGCGATG-1-rep1    1.246474
AAACATGCACACAATT-1-rep1    1.304942
AAACCGAAGTTAACCA-1-rep1    1.424063
AAACCGCGTCAAGTAT-1-rep1    0.638786
AAACCGGCACCATATG-1-rep1    0.111328
                             ...   
TTTGTGAAGGTCAAAG-1-rep2    2.913116
TTTGTGGCAATAATGG-1-rep2    2.834000
TTTGTGGCATTGTGCA-1-rep2    0.847679
TTTGTGTTCGAGGTGG-1-rep2    1.751037
TTTGTGTTCGCATCCT-1-rep2    1.431803
Name: size_factors, Length: 6613, dtype: float64

In [30]:
from scipy.stats import pearsonr

pearsonr(adata_RNA_pca.obs['size_factors'], adata_RNA.obs['size_factors'])

(0.9999999999999999, 0.0)